In [1]:
import pandas as pd
import os
import numpy
from scipy.stats import pearsonr as cor
from math import sqrt
from math import pow
from collections import defaultdict

In [2]:
def getObs_ch1(obsData):
    return pd.read_csv(obsData).loc[:,['CELL_LINE','COMBINATION_ID','SYNERGY_SCORE']].rename(columns={'SYNERGY_SCORE':'OBSERVATION'})

def parCor(u,v,w):
    numerator=cor(u,v)[0] - cor(u,w)[0]*cor(w,v)[0]
    denumerator=sqrt(1-pow(cor(u,w)[0],2)) * sqrt(1-pow(cor(w,v)[0],2))

    return numerator/denumerator

def getGlobalScore_ch1(obs,pred):
    x=obs['OBSERVATION']
    y=pred['PREDICTION']

    z0=obs.groupby('CELL_LINE').transform(numpy.median)['OBSERVATION']
    z1=obs.groupby('COMBINATION_ID').transform(numpy.median)['OBSERVATION']


    numerator=parCor(x,y,z1) - parCor(x,z0,z1)*parCor(z0,y,z1)
    denumerator=sqrt(1-pow(parCor(x,z0,z1),2))*sqrt(1-pow(parCor(z0,y,z1),2))

    return numerator/denumerator

In [64]:
cellLineBool=1
drugCombBool=1
maxconcBool=1
einfBool=1
drugTargetBool=1
mutBool=1
gexBool=0

outputFileName=''
if cellLineBool: outputFileName+='cellID_'
if drugCombBool==1: outputFileName+='drugCombID_'
elif drugCombBool==0: outputFileName+='drugID_'
if maxconcBool: outputFileName+='maxconc_'
if einfBool: outputFileName+='einf_'
if drugTargetBool: outputFileName+='drugTarget_'
if mutBool: outputFileName+='mut_'
if gexBool: outputFileName+='gex_'
outputFileName+='result'
print outputFileName

makeLibfmBool=1

libfmId=0
groupData=[]
groupId=0

if cellLineBool:
    cl={}        
    pr=pd.read_csv('input/ch1_train_combination_and_monoTherapy.csv')
    for i,name in enumerate(set(pr['CELL_LINE'])):
        cl[name]=i
        groupData.append(str(groupId))

    libfmId+=len(cl)
    groupId+=1
    print 'cell line 개수:',len(cl)
    
if drugCombBool==1:
    dc={}
    pr=pd.read_csv('input/ch1_train_combination_and_monoTherapy.csv')
    for i,name in enumerate(set(pr['COMBINATION_ID']),start=libfmId):
        dc[name]=i
        groupData.append(str(groupId))
    libfmId+=len(dc)
    groupId+=1
    print 'drug combination 개수: ',len(dc)
elif drugCombBool==0:
    drugName={}
    pr=pd.read_csv('drug/Drug_info_release.csv')
    for i,name in enumerate(set(pr['ChallengeName']),start=libfmId):
        drugName[name]=i
        groupData.append(str(groupId))
    libfmId+=len(drugName)
    groupId+=1
    print 'drug 개수:',len(drugName)
else:
    print 'no drug'

if maxconcBool:
    maxconca=libfmId
    maxconcb=libfmId+1
    libfmId+=2
    groupData.extend([str(groupId),str(groupId)])
    groupId+=1
    print 'max conc a,b'

if einfBool:
    einfaId=libfmId
    einfbId=libfmId+1
    libfmId+=2
    groupData.extend([str(groupId),str(groupId)])
    groupId+=1
    print 'einf a,b'
    
if drugTargetBool:
    pr=pd.read_csv('drug/woDNA.csv')
    dtl=set() # drug target list
    for targets in pr['Target']:
        if not pd.isnull(targets):
            targetSplit=targets.split(',')
            for i in range(len(targetSplit)):
                if '*' not in targetSplit[i]:
                    dtl.add(targetSplit[i].strip())

    dtId={}
    for i,d0 in enumerate(dtl,start=libfmId):
        dtId[d0]=i
        groupData.append(str(groupId))
    
    ddt={}
    for row in pr.itertuples():
        tmp=set()
        if not pd.isnull(row[2]):
            targetSplit=row[2].split(',')
            for i in range(len(targetSplit)):
                if '*' not in targetSplit[i]:
                    tmp.add(dtId[targetSplit[i].strip()])
        ddt[row[1]]=tmp
    
    print 'drug target 개수: ',len(dtl)
    libfmId+=len(dtl)
    groupId+=1

if mutBool:
    pr=pd.read_csv('drug/woDNA.csv')
    drugTargetSetTmp=set() # drug target list
    for targets in pr['Target']:
        if not pd.isnull(targets):
            targetSplit=targets.split(',')
            for i in range(len(targetSplit)):
                if '*' not in targetSplit[i]:
                    drugTargetSetTmp.add(targetSplit[i].strip())
    
    dtId2={}
    for i,d0 in enumerate(drugTargetSetTmp,start=libfmId):
        dtId2[d0]=i
        groupData.append(str(groupId))    
    
    libfmId+=len(dtId2)
    
    clMut={}
    pr=pd.read_csv('mutation/mutations.csv')
    for row in pr[(pr['Tumour.origin']=='primary')].itertuples():
        if row[1] in dtId2: 
            if row[1]=='BRAF' and row[15]=='p.V600E':
                if row[5] in clMut: clMut[row[5]].add(dtId2['BRAF_V600E'])
                else: clMut[row[5]]=set([dtId2['BRAF_V600E']])
            else:
                if row[5] in clMut: 
                    clMut[row[5]].add(dtId2[row[1]])
                else: clMut[row[5]]=set([dtId2[row[1]]])

    clNotInMut=defaultdict(set)
    for cellLineVar in clMut:
        for drugTargetIdVar in dtId2.values():
            if drugTargetIdVar not in clMut[cellLineVar]:
                clNotInMut[cellLineVar].add(drugTargetIdVar)
        
    print 'mutations 개수: ',len(dtId2)
    groupId+=1

print '총 feature 개수: ',libfmId

if makeLibfmBool:
    trainDir='input/train_set(wo QA limit)2.csv'
    testDir='input/test_set(wo QA limit)2.csv'
    libfmTrain='libfmInput/fmTrain.libfm'
    libfmTest='libfmInput/fmTest.libfm'
    metaDir='libfmInput/meta.txt'
    
    with open(metaDir,'w') as fw:
        for item in groupData:
            fw.write(item+'\n')
    
    def makeLibfmInput(fileDir,libfmFile,score):
        pr=pd.read_csv(fileDir)
        featureString=""

        with open(libfmFile,'w') as fw:
            for row in pr.itertuples():
                if score: featureString+=str(row[12])+' ' # score
                else: featureString+='0 ' # score
                    
                if cellLineBool:
                    if row[1] in cl: featureString+=str(cl[row[1]])+':1 ' # cell line

                if mutBool:
                    if row[1] in clMut:
                        for x in clMut[row[1]]: featureString+=str(x)+':1 '
                    if row[1] in clNotInMut:
                        for x in clNotInMut[row[1]]: featureString+=str(x)+':-1 '

                if drugCombBool==1:
                    if row[14] in dc: featureString+=str(dc[row[14]])+':1 '
                elif drugCombBool==0: featureString+=str(drugName[row[2]])+':1 '+str(drugName[row[3]])+':1 '

                if drugTargetBool:
                    u=ddt[row[2]].intersection(ddt[row[3]])
                    for x in u: featureString+=str(x)+':2 '
                    for x in ddt[row[2]]-u: featureString+=str(x)+':1 '
                    for x in ddt[row[3]]-u: featureString+=str(x)+':1 '
                
                if maxconcBool:
                    featureString+=str(maxconca)+':'+str(row[4]/pr['MAX_CONC_A'].max())+' '
                    featureString+=str(maxconcb)+':'+str(row[5]/pr['MAX_CONC_B'].max())+' '

                if einfBool:
                    featureString+=str(einfaId)+':'+str(row[8]/pr['Einf_A'].max())+' '
                    featureString+=str(einfbId)+':'+str(row[11]/pr['Einf_B'].max())+' '

                featureString=featureString.strip()
                featureString+='\n'
            fw.write(featureString)
        
    makeLibfmInput(trainDir,libfmTrain,True)
    makeLibfmInput(testDir,libfmTest,False)

    def runLibfmTest(i=0,dim='10',iter1='100',stdev='0.001'):
        os.system("ConsoleApplication2 -task r -train "+libfmTrain+" -test "+libfmTest+" -dim '1,1,"+dim
                      +"' -iter "+iter1+" -meta "+metaDir+" -init_stdev "+stdev+" -out prediction\\"+outputFileName+str(i)
                  +" -seed "+str(i))
    
    prTest=pd.read_csv(testDir)
    pred=prTest.loc[:,['CELL_LINE','COMBINATION_ID']]
    obs=getObs_ch1(testDir)
    
    for dim in range(20,21):
        for iter1 in range(270,280,10):
            #print dim,iter1
            for i in range(0,100):
                runLibfmTest(i=i,dim=str(dim),iter1=str(iter1))
                tmp=[]
                with open('prediction/'+outputFileName+str(i),'r') as fr:
                    for row in fr.readlines():
                        tmp.append(float(row.strip()))
            
                pred['PREDICTION']=numpy.asarray(tmp)
                print str(i),getGlobalScore_ch1(obs,pred)

cellID_drugCombID_maxconc_einf_drugTarget_mut_result
cell line 개수: 85
drug combination 개수:  169
max conc a,b
einf a,b
drug target 개수:  96
mutations 개수:  96
총 feature 개수:  450
15 0.238382006105
24 0.249679773885
47 0.208024265678


In [65]:
print 'global',
num=3
a=[]
for i in [15,24,47]:
    tmp=[]
    with open('prediction/'+outputFileName+str(i),'r') as fr:
        for row in fr.readlines():
            tmp.append(float(row.strip()))
    if len(a)!=0:
        a=[sum(x) for x in zip(a, tmp)]
    else:
        a=tmp
a=map(lambda x: x/num, a)
pred['PREDICTION']=numpy.asarray(a)
print getGlobalScore_ch1(obs,pred)

global 0.235109562171


In [4]:
cellLineBool=1
drugCombBool=1
maxconcBool=1
einfBool=1
drugTargetBool=1
mutBool=1
gexBool=0

outputFileName=''
if cellLineBool: outputFileName+='cellID_'
if drugCombBool==1: outputFileName+='drugCombID_'
elif drugCombBool==0: outputFileName+='drugID_'
if maxconcBool: outputFileName+='maxconc_'
if einfBool: outputFileName+='einf_'
if drugTargetBool: outputFileName+='drugTarget_'
if mutBool: outputFileName+='mut_'
if gexBool: outputFileName+='gex_'
outputFileName+='result'
print outputFileName

makeLibfmBool=1

libfmId=0
groupData=[]
groupId=0

if cellLineBool:
    cl={}        
    pr=pd.read_csv('input/ch1_train_combination_and_monoTherapy.csv')
    for i,name in enumerate(set(pr['CELL_LINE'])):
        cl[name]=i
        groupData.append(str(groupId))

    libfmId+=len(cl)
    groupId+=1
    print 'cell line 개수:',len(cl)
    
if drugCombBool==1:
    dc={}
    pr=pd.read_csv('input/ch1_train_combination_and_monoTherapy.csv')
    for i,name in enumerate(set(pr['COMBINATION_ID']),start=libfmId):
        dc[name]=i
        groupData.append(str(groupId))
    libfmId+=len(dc)
    groupId+=1
    print 'drug combination 개수: ',len(dc)
elif drugCombBool==0:
    drugName={}
    pr=pd.read_csv('drug/Drug_info_release.csv')
    for i,name in enumerate(set(pr['ChallengeName']),start=libfmId):
        drugName[name]=i
        groupData.append(str(groupId))
    libfmId+=len(drugName)
    groupId+=1
    print 'drug 개수:',len(drugName)
else:
    print 'no drug'

if maxconcBool:
    maxconca=libfmId
    maxconcb=libfmId+1
    libfmId+=2
    groupData.extend([str(groupId),str(groupId)])
    groupId+=1
    print 'max conc a,b'

if einfBool:
    einfaId=libfmId
    einfbId=libfmId+1
    libfmId+=2
    groupData.extend([str(groupId),str(groupId)])
    groupId+=1
    print 'einf a,b'
    
if drugTargetBool:
    pr=pd.read_csv('drug/woDNA.csv')
    dtl=set() # drug target list
    for targets in pr['Target']:
        if not pd.isnull(targets):
            targetSplit=targets.split(',')
            for i in range(len(targetSplit)):
                if '*' not in targetSplit[i]:
                    dtl.add(targetSplit[i].strip())

    dtId={}
    for i,d0 in enumerate(dtl,start=libfmId):
        dtId[d0]=i
        groupData.append(str(groupId))
    
    ddt={}
    for row in pr.itertuples():
        tmp=set()
        if not pd.isnull(row[2]):
            targetSplit=row[2].split(',')
            for i in range(len(targetSplit)):
                if '*' not in targetSplit[i]:
                    tmp.add(dtId[targetSplit[i].strip()])
        ddt[row[1]]=tmp
    
    print 'drug target 개수: ',len(dtl)
    libfmId+=len(dtl)
    groupId+=1

if mutBool:
    pr=pd.read_csv('drug/woNoise.csv')
    drugTargetSetTmp=set() # drug target list
    for targets in pr['Target']:
        if not pd.isnull(targets):
            targetSplit=targets.split(',')
            for i in range(len(targetSplit)):
                if '*' not in targetSplit[i]:
                    drugTargetSetTmp.add(targetSplit[i].strip())
    
    dtId2={}
    for i,d0 in enumerate(drugTargetSetTmp,start=libfmId):
        dtId2[d0]=i
        groupData.append(str(groupId))    
    
    libfmId+=len(dtId2)
    
    clMut={}
    pr=pd.read_csv('mutation/mutations.csv')
    for row in pr[(pr['Tumour.origin']=='primary')].itertuples():
        if row[1] in dtId2: 
            if row[1]=='BRAF' and row[15]=='p.V600E':
                if row[5] in clMut: clMut[row[5]].add(dtId2['BRAF_V600E'])
                else: clMut[row[5]]=set([dtId2['BRAF_V600E']])
            else:
                if row[5] in clMut: 
                    clMut[row[5]].add(dtId2[row[1]])
                else: clMut[row[5]]=set([dtId2[row[1]]])

    clNotInMut=defaultdict(set)
    for cellLineVar in clMut:
        for drugTargetIdVar in dtId2.values():
            if drugTargetIdVar not in clMut[cellLineVar]:
                clNotInMut[cellLineVar].add(drugTargetIdVar)
        
    print 'mutations 개수: ',len(dtId2)
    groupId+=1

print '총 feature 개수: ',libfmId

if makeLibfmBool:
    trainDir='input/ch1_train_combination_and_monoTherapy.csv'
    testDir='input/ch1_test_monoTherapy.csv'
    libfmTrain='libfmInput/fmTrain.libfm'
    libfmTest='libfmInput/fmTest.libfm'
    metaDir='libfmInput/meta.txt'
    
    with open(metaDir,'w') as fw:
        for item in groupData:
            fw.write(item+'\n')
    
    def makeLibfmInput(fileDir,libfmFile,score):
        pr=pd.read_csv(fileDir)
        featureString=""

        with open(libfmFile,'w') as fw:
            for row in pr.itertuples():
                if score: featureString+=str(row[12])+' ' # score
                else: featureString+='0 ' # score
                    
                if cellLineBool:
                    if row[1] in cl: featureString+=str(cl[row[1]])+':1 ' # cell line

                if mutBool:
                    if row[1] in clMut:
                        for x in clMut[row[1]]: featureString+=str(x)+':1 '
                    if row[1] in clNotInMut:
                        for x in clNotInMut[row[1]]: featureString+=str(x)+':-1 '

                if drugCombBool==1:
                    if row[14] in dc: featureString+=str(dc[row[14]])+':1 '
                elif drugCombBool==0: featureString+=str(drugName[row[2]])+':1 '+str(drugName[row[3]])+':1 '

                if drugTargetBool:
                    u=ddt[row[2]].intersection(ddt[row[3]])
                    for x in u: featureString+=str(x)+':2 '
                    for x in ddt[row[2]]-u: featureString+=str(x)+':1 '
                    for x in ddt[row[3]]-u: featureString+=str(x)+':1 '
                
                if maxconcBool:
                    featureString+=str(maxconca)+':'+str(row[4]/pr['MAX_CONC_A'].max())+' '
                    featureString+=str(maxconcb)+':'+str(row[5]/pr['MAX_CONC_B'].max())+' '

                if einfBool:
                    featureString+=str(einfaId)+':'+str(row[8]/pr['Einf_A'].max())+' '
                    featureString+=str(einfbId)+':'+str(row[11]/pr['Einf_B'].max())+' '

                featureString=featureString.strip()
                featureString+='\n'
            fw.write(featureString)
        
    makeLibfmInput(trainDir,libfmTrain,True)
    makeLibfmInput(testDir,libfmTest,False)

    def runLibfmTest(i=0,dim='10',iter1='100',stdev='0.001'):
        os.system("libfm -task r -train "+libfmTrain+" -test "+libfmTest+" -dim '1,1,"+dim
                      +"' -iter "+iter1+" -meta "+metaDir+" -init_stdev "+stdev+" -out prediction\\"+outputFileName+str(i)
                  +" > log\\log.txt")
        #obs=getObs_ch1(testDir)
        #pred=pd.read_csv('prediction/'+outputFileName+str(i),names=['PREDICTION'])
        #print getGlobalScore_ch1(obs,pred)
    ###################################################################

    prTest=pd.read_csv(testDir)
    pred=prTest.loc[:,['CELL_LINE','COMBINATION_ID']]
    obs=getObs_ch1(testDir)
    
    for x in range(7):
        print x
        for dim in range(20,21):
            for iter1 in range(270,280,10):
                print dim,iter1
                for i in range(10):
                    runLibfmTest(i=i,dim=str(dim),iter1=str(iter1))
                print 'global',
                num=10
                a=[]
                for i in range(num):
                    tmp=[]
                    with open('prediction/'+outputFileName+str(i),'r') as fr:
                        for row in fr.readlines():
                            tmp.append(float(row.strip()))
                    if len(a)!=0:
                        a=[sum(x) for x in zip(a, tmp)]
                    else:
                        a=tmp
                a=map(lambda x: x/num, a)
                pred['PREDICTION']=numpy.asarray(a)

                #pred.to_csv('finalSubmissionFile'+str(dim)+'_'+str(iter1)+'_'+'.csv',index=False)
                print getGlobalScore_ch1(obs,pred)

cellID_drugCombID_maxconc_einf_drugTarget_mut_result
cell line 개수: 85
drug combination 개수:  169
max conc a,b
einf a,b
drug target 개수:  96
mutations 개수:  90
총 feature 개수:  444
0
20 270


KeyboardInterrupt: 

In [1561]:
pr=pd.read_csv('gex/gex_normalized_cl.csv')
tumorSuppressorGenes=['pRb','TP53','PTEN','pVHL','APC','CD95','ST5','YPEL3','ST7','ST14']
for colName in pr.columns.values:
    for tumorGene in tumorSuppressorGenes:
        if tumorGene.lower() in row[1].lower(): print tumorGene,row[1]

In [ ]:
def getprecrec(outfile,testfile):
    fi=open(outfile,'r') #prediction
    arr1=[]
    arr2=[]
    for line in fi:
        if float(line)<0:
            arr1.append(0)
        else:
            arr1.append(1)
    fi.close()
    fi=open(testfile,'r')
    for line in fi:
            arr=line.split(' ')
            if float(arr[0])<0:
                arr2.append(0)
            else:
                arr2.append(1)
    fi.close()
    tp,fp,tn,fn=0,0,0,0
    for x in range(len(arr1)):
        if arr1[x]==arr2[x]:
            if arr1[x]==0:
                    tp+=1
            else:
                    tn+=1
        else:
            if arr1[x]==0:
                    fp+=1
            else:
                    fn+=1
    prec=float(tp)/(float(tp+fp))
    rec=float(tp)/(float(tp+tn))
    print 'PRECISION:\t',prec
    print 'RECALL:\t',rec
    print 'FSCORE:\t',2/((1/prec)+(1/rec))
            
for i in range(10):
    getprecrec('prediction/'+outputFileName+str(i),libfmTest)

In [ ]:
cvTestB=True
if cvTestB:
    fs=[] #feature string
    nr=0 #number of rows

    with open(libfmTrain,'r') as fr:
        for row in fr:
            fs.append(row.strip())
            nr+=1

    subset_size=nr/10
    for i in range(0,10):
        with open(('cv/cvTest'+str(i))+'.libfm','w') as fw:
            if i==9:
                for item in fs[i*subset_size:]:
                    print>>fw, item
            else:
                for item in fs[i*subset_size:][:subset_size]:
                    print>>fw, item
        with open('cv/cvTrain'+str(i)+'.libfm','w') as fw:
            for item in fs[:i*subset_size] + fs[(i+1)*subset_size:]:
                print>> fw, item

In [ ]:
pr=pd.read_csv('ch1_train_combination_and_monoTherapy_shuffled.csv')
pr=pr.loc[pr['QA']==1]
pr.to_csv('monotherapy_shuffled.csv',index=False)

In [ ]:
pr=pd.read_csv('ch1_train_combination_and_monoTherapy_shuffled_test.csv')
pr=pr.loc[pr['QA']==1]
#pr.loc[pr['SYNERGY_SCORE']>100,'SYNERGY_SCORE']=100
#pr.loc[pr['SYNERGY_SCORE']<-100,'SYNERGY_SCORE']=-100
pr.to_csv('monotherapy_shuffled_test.csv',index=False)

In [ ]:
pr=pd.read_csv('methyl/CpG_isle_level/methyl_ilse_beta.csv',index_col=0)
methylSiteSet=set()
for colName in pr.columns.values:
    tmp=pr.sort(colName,ascending=False).head(10).index.values.tolist()
    for methylVal in tmp:
        methylSiteSet.add(methylVal)